In [3]:
import numpy as np
import pandas as pd
import sympy as sp
from sympy import symbols, Function, Eq, dsolve, diff
'''
修改参数：
1.数据的列索引--->修改“y_df.iloc[:, 0]”中的列
2.数据的路径  --->“pd.read_excel(r'C:\Users\86135\Documents\MATLAB\2.xlsx')”
'''
# 读取数据
y_df = pd.read_excel(r'C:\Users\86135\Documents\MATLAB\2.xlsx')  # 读取数据
x0 = y_df.iloc[:, 0].values.astype(float)  # 提取数据列
n = len(x0)

# ================== GM(2,1) 核心算法 ==================
x1 = np.cumsum(x0)           # 累加生成序列
a_x0 = np.diff(x0)           # 累减生成序列
z = 0.5 * (x1[1:] + x1[:-1]) # 均值生成序列

# 构造矩阵 B 和向量 Y
B = np.column_stack((-x0[1:], -z, np.ones(n-1)))
Y = a_x0

# 最小二乘法求解参数并明确命名
coeff, residuals, rank, singular_values = np.linalg.lstsq(B, Y, rcond=None)
a1, a2, b = coeff  # 参数拆分为 a1, a2, b

# ================== 符号求解微分方程 ==================
t = symbols('t')
x = Function('x')(t)
eq = Eq(diff(x, t, t) + a1 * diff(x, t) + a2 * x, b)  # 使用命名参数
ics = {x.subs(t, 0): x1[0], x.subs(t, n-1): x1[-1]}

try:
    sol = dsolve(eq, ics=ics)
    xt = sol.rhs
except Exception as e:
    print("符号求解失败:", e)
    exit()

# ================== 计算拟合值 ==================
t_values = np.arange(n)
x_func = sp.lambdify(t, xt, modules='numpy')
yuce = x_func(t_values)

x0_hat = np.zeros(n)
x0_hat[0] = yuce[0]
for i in range(1, n):
    x0_hat[i] = yuce[i] - yuce[i-1]
x0_hat = np.round(x0_hat).astype(int)

# ================== 结果输出 ==================
print("GM(2,1) 参数: a1={:.4f}, a2={:.4f}, b={:.4f}".format(a1, a2, b))
print("拟合值:", x0_hat)
print("残差:", x0 - x0_hat)
print("相对误差:", np.round(np.abs((x0 - x0_hat) / x0), 6))

GM(2,1) 参数: a1=-1.9951, a2=0.0699, b=-241.5126
拟合值: [130 130 135 140 145 150 155]
残差: [ 0. -5.  5. -5.  5. -5.  5.]
相对误差: [0.       0.04     0.035714 0.037037 0.033333 0.034483 0.03125 ]
